<h1> Data Expansion and Cleaning </h1>

<h3>Contents:</h3>

1) [Exploratory Data Analysis](#EDA) <br>
2) [Creating HSG only dataframe](#HSG_Only) <br>
3) [Categorize data source (Survey or directly given by the school)](#8020) <br>
4) [Calculate relative rank and score, assigning a state change](#Rel_state)<br>
5) [Using Geocoding API to retrieve school-specific geo coodinates](#Geocoding) <br>
6) [Fixing 2021 Data (eventually not used for data visualization)](#2021)

<h2> Exploratory Data Analysis </h2>
<a id="EDA"></a>
<br>
<b> Overview </b>
<br>
The data given was relatively clean with no missing information except those of the 2 and 3 year average. These can be understood for the irregularities in the years which a school participates in ranking. School names were spelled regularly throughout the data source. An assumption we have taken is that the school programs compared for the Financial Times OE Ranking are similar enough to make a fair comparison.  
<br>
<h3> Loading data from source </h3>

In [6]:
import pandas as pd

path = ('FT OE rankings 2007-2020.xlsx')
xl = pd.ExcelFile(path)
print(xl.sheet_names)

['OE 2007-20']


In [108]:
df = pd.read_excel ("FT OE rankings 2007-2020.xlsx")
df

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value
0,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,4
1,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,8.6,4
2,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,7.6,3
3,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,8.7,2
4,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
15115,OE,2007-01-01,50,NaN,Lagos Business School,No,Nigeria,No,No,New skills & learning,8.8,50
15116,OE,2007-01-01,50,NaN,Lagos Business School,No,Nigeria,No,No,Partner schools,3.0,30
15117,OE,2007-01-01,50,NaN,Lagos Business School,No,Nigeria,No,No,Preparation,7.8,50
15118,OE,2007-01-01,50,NaN,Lagos Business School,No,Nigeria,No,No,Quality of participants,7.9,50


In [110]:
#Adding column parameter HSG --> boolean integer
df['HSG']=(df['School name']=='University of St Gallen').astype(int)

In [111]:
#display table
df

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG
0,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,4,0
1,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,8.6,4,0
2,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,7.6,3,0
3,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,8.7,2,0
4,OE,2020-01-01,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,OE,2007-01-01,50,NaN,Lagos Business School,No,Nigeria,No,No,New skills & learning,8.8,50,0
15116,OE,2007-01-01,50,NaN,Lagos Business School,No,Nigeria,No,No,Partner schools,3.0,30,0
15117,OE,2007-01-01,50,NaN,Lagos Business School,No,Nigeria,No,No,Preparation,7.8,50,0
15118,OE,2007-01-01,50,NaN,Lagos Business School,No,Nigeria,No,No,Quality of participants,7.9,50,0


In [112]:
print(df['Ranking criteria'].unique())

ranking_criterias = df['Ranking criteria'].unique()

['Aims achieved' 'Course design' 'Facilities' 'Faculty'
 'Faculty diversity' 'Female participants' 'Follow-up'
 'Food & accommodation' 'Growth' 'International location'
 'International participants' 'New skills & learning' 'Partner schools'
 'Preparation' 'Quality of participants' 'Teaching methods & materials']


In [113]:
#convert Year to numerical year only
df['Year'] = df['Year'].dt.year

In [114]:
# realise different schools rank different criterias differently
by_year_hsg = df[df["School name"]=='University of St Gallen'].groupby(df['Year'])

In [115]:
(by_year_hsg)

<a id="HSG_Only"></a>
<h3>Create column 'HSG' of values 0 and 1</h3>
  

In [116]:
hsg = df[df["School name"]=='University of St Gallen']
hsg

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG
400,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,20,1
401,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Course design,8.6,29,1
402,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Facilities,7.6,18,1
403,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Faculty,8.7,31,1
404,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14699,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,New skills & learning,8.8,25,1
14700,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Partner schools,3.0,10,1
14701,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Preparation,7.8,9,1
14702,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Quality of participants,7.9,24,1


In [117]:
import numpy as np
hsg.groupby(['Year'], as_index=False)['Ranking weight']

<a id="8020"></a>
<h3> Create new column to categorize data given by school and participants (values in 0 or 1)</h3>

In [118]:
#label boolean 1/0 for in 20p -> provided by the school
#80% provided by the participants 

#list of criterias provided by school
criteria_20p = ["Female participants","International participants", 
                "International location", "Growth", "Partner schools", "Faculty diversity"]



In [119]:
def categorize_source(x):
    if x in criteria_20p:
        return 1
    else:
        return 0

In [120]:
# df[df["Ranking criteria"] in criteria_20p]
df['20p'] =  df['Ranking criteria'].apply(categorize_source)

In [121]:
df

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG,20p
0,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,4,0,0
1,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,8.6,4,0,0
2,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,7.6,3,0,0
3,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,8.7,2,0,0
4,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,New skills & learning,8.8,50,0,0
15116,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Partner schools,3.0,30,0,1
15117,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Preparation,7.8,50,0,0
15118,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Quality of participants,7.9,50,0,0


In [25]:
df.to_excel("OE_v1.xlsx")

<a id="Rel_state"></a>
<h3>Create new column 'Relative value' to give the relative score per ranking criteria for each year</h3>

The relative rank or score was calculated per ranking criteria, per year for a year on year comparison for each ranking criteria, and each school. The algorithm is as follows.  
 
By grouping the data points by the year and calculating the number of unique school names in each year, we found that there are different number of schools in each year.  

The relative rank is calculated by dividing the rank with the total number of unique school names in that year to give a value between 0 and 1.

In [122]:
num_schools_arr = df.groupby(["Year"]).nunique()["School name"]

In [123]:
num_schools_arr

Year
2007    50
2008    50
2009    55
2010    60
2011    65
2012    65
2013    70
2014    70
2015    75
2016    75
2017    75
2018    80
2019    80
2020    75
Name: School name, dtype: int64

Notice that each year has a different number of competing schools, we need to be careful of each school in the dataset for its school name (spelling errors) and that different years have a different set of participating schools.

In [125]:
num_schools_arr[df.loc[9]["Year"]] 

#cross checking with visual evaluation of dataset

75

In [126]:
rel = []
for i in range(0,len(df)):
    res = df["Value"][i]/num_schools_arr[df.loc[i]["Year"]]
    rel.append(res)

rel

[0.05333333333333334,
 0.05333333333333334,
 0.04,
 0.02666666666666667,
 0.06666666666666667,
 0.36,
 0.10666666666666667,
 0.02666666666666667,
 0.38666666666666666,
 0.49333333333333335,
 0.08,
 0.04,
 0.5733333333333334,
 0.09333333333333334,
 0.05333333333333334,
 0.013333333333333334,
 0.04,
 0.04,
 0.12,
 0.04,
 0.09333333333333334,
 0.48,
 0.04,
 0.22666666666666666,
 0.4666666666666667,
 0.48,
 0.12,
 0.06666666666666667,
 0.21333333333333335,
 0.06666666666666667,
 0.02666666666666667,
 0.05333333333333334,
 0.08,
 0.10666666666666667,
 0.06666666666666667,
 0.08,
 0.05333333333333334,
 0.44,
 0.13333333333333333,
 0.10666666666666667,
 0.41333333333333333,
 0.28,
 0.05333333333333334,
 0.22666666666666666,
 0.3466666666666667,
 0.13333333333333333,
 0.04,
 0.06666666666666667,
 0.02666666666666667,
 0.013333333333333334,
 0.013333333333333334,
 0.013333333333333334,
 0.56,
 0.76,
 0.26666666666666666,
 0.14666666666666667,
 0.5066666666666667,
 0.26666666666666666,
 0.426666

In [127]:
df["Relative_value"] = rel

In [128]:
df

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG,20p,Relative_value
0,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,4,0,0,0.053333
1,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,8.6,4,0,0,0.053333
2,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,7.6,3,0,0,0.040000
3,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,8.7,2,0,0,0.026667
4,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,5,0,1,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,New skills & learning,8.8,50,0,0,1.000000
15116,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Partner schools,3.0,30,0,1,0.600000
15117,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Preparation,7.8,50,0,0,1.000000
15118,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Quality of participants,7.9,50,0,0,1.000000


<b>Mark the status by calculating the percentage change, assigning it to column 'state' and categorizing the 'State change' value by 0, 1 and -1.</b><br>
We also calculate the percentage change of that relative rank with its previous years and assign it a ‘state change’ to reflect a positive, neutral or negative change. These are denoted by 1, 0 and -1 respectively in our extended dataset. As some schools are not in the participating schools list each year, the inconsistency creates some empty data points for this attribute. A state of 0 or neutral has been assigned to these to indicate that they did not change from its previous year. This is an assumption made that the value did not change for the year that a school did not participate in the Financial Times OE Ranking. 

In [137]:
df["state"] = df.sort_values('Year').groupby(
    ['Ranking criteria', 'School name']).Relative_value.pct_change()
df['state']= df['state'].fillna(value=0)
df

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG,20p,Relative_value,state
0,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,4,0,0,0.053333,-0.466667
1,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,8.6,4,0,0,0.053333,-0.466667
2,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,7.6,3,0,0,0.040000,-0.600000
3,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,8.7,2,0,0,0.026667,0.066667
4,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,5,0,1,0.066667,1.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,New skills & learning,8.8,50,0,0,1.000000,0.000000
15116,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Partner schools,3.0,30,0,1,0.600000,0.000000
15117,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Preparation,7.8,50,0,0,1.000000,0.000000
15118,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Quality of participants,7.9,50,0,0,1.000000,0.000000


In [138]:
df[df["School name"]=='University of St Gallen']

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG,20p,Relative_value,state
400,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,20,1,0,0.266667,0.254902
401,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Course design,8.6,29,1,0,0.386667,0.344928
402,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Facilities,7.6,18,1,0,0.240000,-0.085714
403,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Faculty,8.7,31,1,0,0.413333,0.033333
404,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,30,1,1,0.400000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14699,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,New skills & learning,8.8,25,1,0,0.500000,0.000000
14700,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Partner schools,3.0,10,1,1,0.200000,0.000000
14701,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Preparation,7.8,9,1,0,0.180000,0.000000
14702,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Quality of participants,7.9,24,1,0,0.480000,0.000000


In [140]:
def categorize_state(x):
    if x>0:
        return 1
    elif x==0:
        return 0
    else:
        return -1

In [143]:
df["state_change"]=df['state'].apply(categorize_state)
df



,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG,20p,Relative_value,state,state_change
0,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,4,0,0,0.053333,-0.466667,-1
1,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,8.6,4,0,0,0.053333,-0.466667,-1
2,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,7.6,3,0,0,0.040000,-0.600000,-1
3,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,8.7,2,0,0,0.026667,0.066667,1
4,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,5,0,1,0.066667,1.666667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,New skills & learning,8.8,50,0,0,1.000000,0.000000,0
15116,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Partner schools,3.0,30,0,1,0.600000,0.000000,0
15117,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Preparation,7.8,50,0,0,1.000000,0.000000,0
15118,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Quality of participants,7.9,50,0,0,1.000000,0.000000,0


In [ ]:
# df.to_excel("OE_v2.xlsx")

### EDA Continued


In [89]:
hsg.sort_values(['Year'],ascending=True).groupby(['Year','Ranking criteria',"School name"]).head()

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG
14703,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Teaching methods & materials,8.3,26,1
14688,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Aims achieved,8.6,19,1
14689,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Course design,8.7,27,1
14690,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Facilities,7.4,27,1
14691,OE,2007,23,NaN,University of St Gallen,Yes,Switzerland,Yes,Yes,Faculty,9.0,28,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Preparation,7.7,11,1
414,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Quality of participants,7.9,56,1
415,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Teaching methods & materials,8.3,24,1
408,OE,2020,26,26.0,University of St Gallen,Yes,Switzerland,Yes,Yes,Growth,5.0,71,1


In [ ]:
# Assumption 1: the ranking weight aggregates with a 3 year average or 2 year average

In [ ]:
#data viz 1: "ranking weight and 20p": target the more influential data. e.g. participant response --> invest more to improve perception and experience

In [ ]:
#finding out why HSG is ranked lower/higher

# assumption 2: is the program compared similar enough to make a comparison? --> yes, already similar enough

<a id="Geocoding"></a>
<h3>Including Geocoding for more specific coordinates</h3>
<br>
This was done with three attempts with three different APIs. The one we eventually used was the Google Geocoding API. The two others were GeoCodio and Open Street Maps. The reason that GeoCodio and Open Street Maps were not suitable were that the school names were not able to successfully invoke a request for the geo-coordinates. In other words, they are not robust enough to accept different forms of a school name and correct or redirect it like the Google API can. We attempted changing the school names and cleaning it with regex when using the other 2 APIs but realized it was not feasible to do so for all the schools that faced such a problem. For Open Street Maps, more than half of the school names required modification, manually. With Google Maps, we only needed to modify about 5 school names. Though it still required some manual modification, it was not as time consuming.  

<h4>[Attempt, Method 1] Using GeoCodio</h4>

In [27]:
geocode_api_key = 

In [158]:
# !pip install pygeocodio
from geocodio import GeocodioClient
client = GeocodioClient(geocode_api_key, timeout = 15)
geocoded_location = client.geocode("University of Oxford: Saïd")
geocoded_location.coords

# locations = client.batch_geocode(df['School name'].unique())

(39.521163, -84.376804)

In [153]:
df['School name'].unique()

array(['IMD', 'University of Oxford: Saïd', 'Insead',
       'University of Michigan: Ross',
       'Stanford Graduate School of Business',
       'University of Chicago: Booth',
       'University of Pennsylvania: Wharton', 'HEC Paris', 'ESMT Berlin',
       'Iese Business School', 'Columbia Business School',
       'UCLA Anderson School of Management', 'Fundação Dom Cabral',
       'Washington University: Olin', 'Esade Business School',
       'Essec Business School', 'University of Toronto: Rotman',
       'Stockholm School of Economics', 'University of Cambridge: Judge',
       'Western University: Ivey', 'London Business School',
       'Henley Business School', 'Hult Ashridge Executive Education',
       'Kaist College of Business', 'SDA Bocconi School of Management',
       'University of St Gallen', 'Nyenrode Business Universiteit',
       "Queen's University: Smith",
       'Shanghai Jiao Tong University: Antai', 'Ceibs',
       'Vlerick Business School', 'Cranfield School of 

In [171]:
!pip3 install urllib3
import urllib3

address="University of Queensland Business School"
key=""
url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (address.replace(' ','+'), key)

response = urllib3.urlopen(url)

jsongeocode = response.read()

AttributeError: module 'urllib3' has no attribute 'urlopen'

<h4>[Used in Final] Using Google API</h4>
<br>
With the Python geopy and googlemaps libraries, we could also retrieve its address and we have hence added the address together with the longitude and latitude. In order to ensure we do not change the original school name in the dataset, a new column titled “reformat_name” was created for the reformatted school name that can successfully generate a positive return from the API. This was manually tested on Google Maps.  

In [167]:
!pip install geopy
!pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38531 sha256=15cbcc30ced2ed3b7836dc51a4b68f13b48136ec4291a431d56ffca0d107143c
  Stored in directory: c:\users\andrew ng\appdata\local\pip\cache\wheels\2d\e9\ee\336c1792ed04efbf4fe9c89662d701f5c1f1fdddf8782951d3
Successfully built googlemaps


In [168]:
from geopy.geocoders import GoogleV3
import geopy.distance
import googlemaps

In [287]:
API="" #created own api key
geolocator = GoogleV3(api_key=API)

print(type(geolocator))

<class 'geopy.geocoders.google.GoogleV3'>


In [288]:
name = 'University of St Gallen' 
location = geolocator.geocode(name)

print(location.address)
print(location.latitude, location.longitude)

Dufourstrasse 50, 9000 St. Gallen, Switzerland
47.4321575 9.3755673


In [289]:
gmaps_key = googlemaps.Client(key=API)

df_schnames = pd.DataFrame(df['School name'].unique())

In [290]:
df_schnames

,0
0,IMD
1,University of Oxford: Saïd
2,Insead
3,University of Michigan: Ross
4,Stanford Graduate School of Business
...,...
101,Wisconsin School of Business
102,University of Queensland Business School
103,Duke University: Fuqua
104,Babson Executive Education


In [291]:
df_schnames=df_schnames.rename(index=str, columns={0:'School name'})

In [335]:
g=gmaps_key.geocode('ESAN Graduate School of Management and Business')
lat = g[0]["geometry"]["location"]["lat"]
long = g[0]["geometry"]["location"]["lng"]
print('Latitude: '+str(lat)+', Longitude: '+str(long))

Latitude: 38.5347704, Longitude: -121.7473888


In [304]:
g[0]

{'address_components': [{'long_name': '50',
   'short_name': '50',
   'types': ['street_number']},
  {'long_name': 'Dufourstrasse',
   'short_name': 'Dufourstrasse',
   'types': ['route']},
  {'long_name': 'St. Gallen',
   'short_name': 'St. Gallen',
   'types': ['locality', 'political']},
  {'long_name': 'Sankt Gallen',
   'short_name': 'St. Gallen',
   'types': ['administrative_area_level_2', 'political']},
  {'long_name': 'Sankt Gallen',
   'short_name': 'SG',
   'types': ['administrative_area_level_1', 'political']},
  {'long_name': 'Switzerland',
   'short_name': 'CH',
   'types': ['country', 'political']},
  {'long_name': '9000', 'short_name': '9000', 'types': ['postal_code']}],
 'formatted_address': 'Dufourstrasse 50, 9000 St. Gallen, Switzerland',
 'geometry': {'location': {'lat': 47.4321575, 'lng': 9.3755673},
  'location_type': 'ROOFTOP',
  'viewport': {'northeast': {'lat': 47.4338484, 'lng': 9.3774762},
   'southwest': {'lat': 47.4297395, 'lng': 9.373839799999999}}},
 'parti

In [337]:
def geocode_addr(add):
    try:
        g = gmaps_key.geocode(add)
        addr = g[0]["formatted_address"]
        return addr
    except:
        return ""

df_schnames['address'] = df_schnames['reformat_name'].apply(geocode_addr)

In [338]:
#checking for empty data
df_schnames[df_schnames['address']==""]

,School name,address,reformat_name


In [336]:
df_schnames['reformat_name']=df_schnames['School name']
df_schnames['reformat_name']= df_schnames['reformat_name'].replace(
    ['Esan',
     'Fundação Instituto de Administração',
     'Sun Yat-sen Business School'],
    ['ESAN Graduate School of Management and Business',
     'Institute of Management Foundation',
     'School of Business Sun Yat-Sen University'])
# df_schnames[df_schnames['School name']=='Esan']

In [339]:
def geocode_lat(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    return lat
def geocode_lon(add):
    g = gmaps_key.geocode(add)
    lng = g[0]["geometry"]["location"]["lng"]
    return lng

df_schnames['glatitude'] = df_schnames['reformat_name'].apply(
    geocode_lat)
df_schnames['glongitude'] = df_schnames['reformat_name'].apply(
    geocode_lon)
df_schnames.isnull().values.any() 

False

In [340]:
df_schnames

,School name,address,reformat_name,glatitude,glongitude
0,IMD,"Chem. de Bellerive 23, 1003 Lausanne, Switzerland",IMD,46.510410,6.621950
1,University of Oxford: Saïd,"Park End St, Oxford OX1 1HP, UK",University of Oxford: Saïd,51.753252,-1.268270
2,Insead,"Bd de Constance, 77300 Fontainebleau, France",Insead,48.405113,2.685654
3,University of Michigan: Ross,"701 Tappan Ave, Ann Arbor, MI 48109, USA",University of Michigan: Ross,42.272969,-83.737863
4,Stanford Graduate School of Business,"655 Knight Way, Stanford, CA 94305, USA",Stanford Graduate School of Business,37.428052,-122.161173
...,...,...,...,...,...
101,Wisconsin School of Business,"975 University Ave, Madison, WI 53706, USA",Wisconsin School of Business,43.072709,-89.401636
102,University of Queensland Business School,"Colin Clark, 39 Blair Dr, St Lucia QLD 4067, A...",University of Queensland Business School,-27.494711,153.014247
103,Duke University: Fuqua,"100 Fuqua Drive, Durham, NC 27708, USA",Duke University: Fuqua,35.998547,-78.945901
104,Babson Executive Education,"231 Forest St, Babson Park, MA 02457, USA",Babson Executive Education,42.296442,-71.265708


In [352]:
df_oe_v2 = pd.read_excel('OE_v2.xlsx', index_col =0)
df_oe_v2

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG,20p,Relative_value,state,state_change
0,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,4,0,0,0.053333,-0.466667,-1
1,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,8.6,4,0,0,0.053333,-0.466667,-1
2,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,7.6,3,0,0,0.040000,-0.600000,-1
3,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,8.7,2,0,0,0.026667,0.066667,1
4,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,5,0,1,0.066667,1.666667,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,New skills & learning,8.8,50,0,0,1.000000,0.000000,0
15116,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Partner schools,3.0,30,0,1,0.600000,0.000000,0
15117,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Preparation,7.8,50,0,0,1.000000,0.000000,0
15118,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Quality of participants,7.9,50,0,0,1.000000,0.000000,0


In [376]:
df_schnames.loc[df_schnames['School name'] == 'Lagos Business School']['address'][0]

'Lekki - Epe Expy, Aja 106104, Lagos, Nigeria'

In [378]:
school_list = df_oe_v2["School name"].to_list()
full_addresses = []
latitudes = []
longitudes = []
for school_name in school_list:
    full_addresses.append(str(df_schnames.loc[df_schnames['School name'] == school_name]['address'][0]))
    latitudes.append(str(df_schnames.loc[df_schnames['School name'] == school_name]['glatitude'][0]))
    longitudes.append(str(df_schnames.loc[df_schnames['School name'] == school_name]['glongitude'][0]))
len(full_addresses)

15120

In [381]:
df_oe_v2['full_address']=full_addresses
df_oe_v2['latitude']=latitudes
df_oe_v2['longitude']=longitudes
df_oe_v2

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,...,Value,HSG,20p,Relative_value,state,state_change,full_addresses,full_address,latitude,longitude
0,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,...,4,0,0,0.053333,-0.466667,-1,"Chem. de Bellerive 23, 1003 Lausanne, Switzerland","Chem. de Bellerive 23, 1003 Lausanne, Switzerland",46.5104105,6.6219502
1,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,...,4,0,0,0.053333,-0.466667,-1,"Chem. de Bellerive 23, 1003 Lausanne, Switzerland","Chem. de Bellerive 23, 1003 Lausanne, Switzerland",46.5104105,6.6219502
2,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,...,3,0,0,0.040000,-0.600000,-1,"Chem. de Bellerive 23, 1003 Lausanne, Switzerland","Chem. de Bellerive 23, 1003 Lausanne, Switzerland",46.5104105,6.6219502
3,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,...,2,0,0,0.026667,0.066667,1,"Chem. de Bellerive 23, 1003 Lausanne, Switzerland","Chem. de Bellerive 23, 1003 Lausanne, Switzerland",46.5104105,6.6219502
4,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,...,5,0,1,0.066667,1.666667,1,"Chem. de Bellerive 23, 1003 Lausanne, Switzerland","Chem. de Bellerive 23, 1003 Lausanne, Switzerland",46.5104105,6.6219502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15115,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,New skills & learning,...,50,0,0,1.000000,0.000000,0,"Lekki - Epe Expy, Aja 106104, Lagos, Nigeria","Lekki - Epe Expy, Aja 106104, Lagos, Nigeria",6.472829399999999,3.603556299999999
15116,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Partner schools,...,30,0,1,0.600000,0.000000,0,"Lekki - Epe Expy, Aja 106104, Lagos, Nigeria","Lekki - Epe Expy, Aja 106104, Lagos, Nigeria",6.472829399999999,3.603556299999999
15117,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Preparation,...,50,0,0,1.000000,0.000000,0,"Lekki - Epe Expy, Aja 106104, Lagos, Nigeria","Lekki - Epe Expy, Aja 106104, Lagos, Nigeria",6.472829399999999,3.603556299999999
15118,OE,2007,50,NaN,Lagos Business School,No,Nigeria,No,No,Quality of participants,...,50,0,0,1.000000,0.000000,0,"Lekki - Epe Expy, Aja 106104, Lagos, Nigeria","Lekki - Epe Expy, Aja 106104, Lagos, Nigeria",6.472829399999999,3.603556299999999


In [382]:
df_oe_v2.to_excel('df_oe_v3_geoinfo.xlsx')

In [383]:
df_schnames.to_excel('University_Geolocation.xlsx')

In [384]:
df_schnames

,School name,address,reformat_name,glatitude,glongitude
0,IMD,"Chem. de Bellerive 23, 1003 Lausanne, Switzerland",IMD,46.510410,6.621950
1,University of Oxford: Saïd,"Park End St, Oxford OX1 1HP, UK",University of Oxford: Saïd,51.753252,-1.268270
2,Insead,"Bd de Constance, 77300 Fontainebleau, France",Insead,48.405113,2.685654
3,University of Michigan: Ross,"701 Tappan Ave, Ann Arbor, MI 48109, USA",University of Michigan: Ross,42.272969,-83.737863
4,Stanford Graduate School of Business,"655 Knight Way, Stanford, CA 94305, USA",Stanford Graduate School of Business,37.428052,-122.161173
...,...,...,...,...,...
101,Wisconsin School of Business,"975 University Ave, Madison, WI 53706, USA",Wisconsin School of Business,43.072709,-89.401636
102,University of Queensland Business School,"Colin Clark, 39 Blair Dr, St Lucia QLD 4067, A...",University of Queensland Business School,-27.494711,153.014247
103,Duke University: Fuqua,"100 Fuqua Drive, Durham, NC 27708, USA",Duke University: Fuqua,35.998547,-78.945901
104,Babson Executive Education,"231 Forest St, Babson Park, MA 02457, USA",Babson Executive Education,42.296442,-71.265708


#### Using open street maps (not used in final)


In [195]:
# !pip install url
# !pip install requests

In [225]:
df_schnames['enc_add'].head(106)

0                                           IMD
1                     University of Oxford Saïd
2                                        Insead
3                   University of Michigan Ross
4          Stanford Graduate School of Business
                         ...                   
101                Wisconsin School of Business
102    University of Queensland Business School
103                       Duke University Fuqua
104                  Babson Executive Education
105     Macquarie Graduate School of Management
Name: enc_add, Length: 106, dtype: object

In [244]:
import re
def clean_add(x):
    if "ï" in x:
        x.replace('ï','i')
    return re.sub(r'[^\w\s]', '', str(x))
df_schnames['enc_add'] = df_schnames['School name'].apply(clean_add)
df_schnames[df_schnames['School name']== "University of Oxford: Saïd"] ="Saïd Business School"
df_schnames[df_schnames['School name']== "University of Michigan: Ross"] ='Michigan Ross Business School'
df_schnames

,School name,enc_add
0,IMD,IMD
1,Saïd Business School,Saïd Business School
2,Insead,Insead
3,Michigan Ross Business School,Michigan Ross Business School
4,Stanford Graduate School of Business,Stanford Graduate School of Business
...,...,...
101,Wisconsin School of Business,Wisconsin School of Business
102,University of Queensland Business School,University of Queensland Business School
103,Duke University: Fuqua,Duke University Fuqua
104,Babson Executive Education,Babson Executive Education


In [241]:
import urllib
import requests


url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(df_schnames['enc_add'][1]) +'?format=json'
response = requests.get(url).json()
print('Latitude: '+response[0]['lat']+', Longitude: '+response[0]['lon'])

Latitude: 51.7537125, Longitude: -1.2684695174759821


In [217]:
# url = 'https://nominatim.openstreetmap.org/search/' 

# def geocode_lat(add):
#     url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(add) +'?format=json'
#     lat = requests.get(url).json()[0]['lat']
#     return lat
# def geocode_lon(add):
#     url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(add) +'?format=json'
#     lng = requests.get(url).json()[0]['lon']
#     return lng

# df_schnames['glatitude'] = df_schnames['School name'].apply(geocode_lat)
# df_schnames['glongitude'] = df_schnames['School name'].apply(geocode_lon)

In [284]:
fix = []
can = 0

for i in range(106):
    add = df_schnames['enc_add'][i]
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(df_schnames['enc_add'][i] ) +'?format=json'
    response = requests.get(url).json()
    try: 
        lat = response[0]['lat']        
#         print('Latitude: '+response[0]['lat']+', Longitude: '+response[0]['lon'])
        can +=1

    except: 
        fix.append(add)

In [281]:
fix

['University of Chicago Booth',
 'Washington University Olin',
 'Stockholm School of Economics',
 'University of Cambridge Judge',
 'Western University Ivey',
 'Hult Ashridge Executive Education',
 'Kaist College of Business',
 'Queens University Smith',
 'Shanghai Jiao Tong University Antai',
 'Gordon Institute of Business Science at UP',
 'NHH Norwegian School of Economics',
 'AGSM at UNSW Business School',
 'National University of Singapore Business School',
 'Thunderbird School of Global Management at ASU',
 'University College Dublin Smurfit',
 'Universidad de los Andes School of Management',
 'York University Schulich',
 'Eada Business School Barcelona',
 'University of British Columbia Sauder',
 'Frankfurt School of Finance and Management',
 'University of Arizona Eller',
 'Rotterdam School of Management Erasmus University',
 'IAE Business School',
 'Florida Atlantic University College of Business',
 'Moscow School of Management Skolkovo',
 'EMLyon Business School',
 'BI Norwegi

In [285]:
can #with sch of biz

0

In [283]:
len(fix) + can == 106

True

In [203]:
url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(df_schnames['School name'][0]) +'?format=json'
response = requests.get(url).json()
response[0]['']

{'place_id': 282412974,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 1762061,
 'boundingbox': ['46.5089527', '46.5114253', '6.6185034', '6.6232768'],
 'lat': '46.5099312',
 'lon': '6.6214752240252075',
 'display_name': 'IMD, 23, Chemin de Bellerive, Lausanne, District de Lausanne, Vaud, 1007, Schweiz/Suisse/Svizzera/Svizra',
 'class': 'amenity',
 'type': 'school',
 'importance': 0.4798815985584707,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//education_school.p.20.png'}

<a id="2021"></a>
<h3>Fixing 2021 data</h3><br>
We found data of more performance markers for each university on the Financial Times Ranking website but they were only found for 2021. The rankings had also not yet been evaluated and announced on the site. Upon further exploratory work, it was also found that there are schools in the 2021 list that are not found in the previous year’s list of participating schools. Each school name was also spelled differently from the way it was spelled in the given dataset. In our correction of this data to prepare it for reasonable comparison with 2020 data, we first manually corrected the spelling of each school name that had a different spelling or wording. Next, we used the 2020 ranking to assign to the 2021 data by using inner join of the 2021 data set and a table of the school’s rankings in 2020. In essence, we also dropped the schools that appeared in 2021 but not in 2020. However, given that these data points were insufficient for a comparison analysis, we did not end up using this extended dataset. 

Additionally, using excel LOOKUP() functions and exchange rates found on the internet, currency values were standardized to that of CHF. This can be found in the 2021 Metadata FT OE.xlsx file. 

 

In [30]:
#loading data
import pandas as pd
df_2021 = pd.read_excel("2021 Metadata FT OE.xlsx", index_col =0)

In [31]:
df_2021 = df_2021.rename(columns = {'School Name':'School name'})
df_2021

,School name,Location (main campus),Unnamed: 3,Tuition cost of GMP,Unnamed: 5,Tuition cost of AMP,Unnamed: 7,Revenue,Repeat business (% of revenue),No. of GMP progs,...,Progs in p'ship (orgs/co),Largest flagship GMP,"Avg teaching hours, GMP",Women on GMP (%),"GMP attendees, 2020",Largest flagship AMP,Unnamed: 23,"Avg teaching hours, AMP",Women on AMP (%),"AMP attendees, 2020"
Year,,,,,,,,,,,,,,,,,,,,,
2021,UCLA Anderson School of Management,US,USD,"$20,000",NaN,"$7,500",NaN,D,83.0,3,...,4,Executive Program,186,26.0,57,Corporate Governance Program,NaN,33.0,70.0,17
2021,Western University: Ivey,Canada,CAD,"C$3,500",NaN,"C$8,000",NaN,D,38.0,2,...,7,Ivey Performance Management Program,25,72.0,45,Ivey Leadership Program,NaN,40.0,53.0,58
2021,Incae Business School,Costa Rica,CRC,"$13,600",NaN,"$9,000",NaN,C,10.0,4,...,1,Programa de Alta Gerencia,160,20.0,81,Senior Executive Program,NaN,50.0,30.0,24
2021,Eada Business School Barcelona,Spain,EUR,"€4,500",NaN,"€4,000",NaN,C,35.0,4,...,8,Sustainability & Innovation Program,60,39.0,120,International Leadership Programme,NaN,60.0,39.0,193
2021,Politecnico di Milano School of Management,Italy,EUR,"€7,000",NaN,"€5,000",NaN,C,12.0,9,...,8,Executive Program in Digital Transformation,128,6.0,34,Professional GBS Certification Program,NaN,36.0,26.0,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,Nyenrode Business Universiteit,Netherlands,NaN,"€14,995",NaN,"€13,995",NaN,B,25.0,8,...,40,Leadership Development Program,104,25.0,40,Nyenrode Commissarissencyclus 45-46-48,NaN,144.0,33.0,121
2021,Iese Business School,Spain,NaN,"€28,400",NaN,"€28,400",NaN,A,40.0,36,...,13,Programa de Desarrollo Directivo (PDD),203,31.0,"1,354",Programa de Dirección General (PDG),NaN,221.0,31.0,"1,173"
2021,Mannheim Business School,Germany,NaN,"€7,500",NaN,"€9,000",NaN,D,5.0,1,...,0,General Management Analytics Certificate,300,38.0,75,Mannheim Agile Leadership Certificate,NaN,96.0,36.0,52


In [17]:
df_for_rankings = pd.read_excel("OE_v1.xlsx",index_col =0)
df_2020 = df_for_rankings[df_for_rankings['Year']==2020]
df_2020

,Ranking,Year,Rank,3-year average,School name,EBS 2019 TOP 20,Country,Europe,DACH,Ranking criteria,Ranking weight,Value,HSG,20p
0,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Aims achieved,8.4,4,0,0
1,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Course design,8.6,4,0,0
2,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Facilities,7.6,3,0,0
3,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty,8.7,2,0,0
4,OE,2020,1,1.0,IMD,Yes,Switzerland,Yes,Yes,Faculty diversity,5.0,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,OE,2020,75,73.0,Porto Business School,No,Portugal,Yes,No,New skills & learning,8.7,75,0,0
1196,OE,2020,75,73.0,Porto Business School,No,Portugal,Yes,No,Partner schools,3.0,28,0,1
1197,OE,2020,75,73.0,Porto Business School,No,Portugal,Yes,No,Preparation,7.7,74,0,0
1198,OE,2020,75,73.0,Porto Business School,No,Portugal,Yes,No,Quality of participants,7.9,72,0,0


In [32]:
try1 = df_2020[['School name','Rank']].drop_duplicates()
try1.groupby('School name')['School name'].count()

School name
AGSM at UNSW Business School                       1
Aalto University                                   1
Alberta School of Business                         1
American University in Cairo School of Business    1
Audencia Business School                           1
                                                  ..
Vlerick Business School                            1
WHU – Otto Beisheim School of Management           1
Washington University: Olin                        1
Western University: Ivey                           1
York University: Schulich                          1
Name: School name, Length: 75, dtype: int64

In [33]:
try1[try1['School name']=='IMD']

,School name,Rank
0,IMD,1


In [34]:
try1

,School name,Rank
0,IMD,1
16,University of Oxford: Saïd,2
32,Insead,3
48,University of Michigan: Ross,3
64,Stanford Graduate School of Business,5
...,...,...
1120,Kedge Business School,71
1136,Católica Lisbon School of Business and Economics,72
1152,Solvay Brussels School of Economics and Manage...,73
1168,Politecnico di Milano School of Management,74


In [35]:
try_inner = pd.merge(try1,df_2021, on='School name', how = 'inner')
try_inner

,School name,Rank,Location (main campus),Unnamed: 3,Tuition cost of GMP,Unnamed: 5,Tuition cost of AMP,Unnamed: 7,Revenue,Repeat business (% of revenue),...,Progs in p'ship (orgs/co),Largest flagship GMP,"Avg teaching hours, GMP",Women on GMP (%),"GMP attendees, 2020",Largest flagship AMP,Unnamed: 23,"Avg teaching hours, AMP",Women on AMP (%),"AMP attendees, 2020"
0,IMD,1,Switzerland,NaN,"SFr13,500",NaN,"SFr30,000",NaN,A,40.0,...,2,High Performance Leadership,48,26.0,212,Advanced Management Program,NaN,150.0,16.0,19
1,University of Oxford: Saïd,2,UK,NaN,"£8,900",NaN,"£18,500",NaN,A,13.0,...,9,Oxford High Performance Leadership Programme,58,35.0,219,Oxford Advanced Management and Leadership Prog...,NaN,202.0,23.0,73
2,Insead,3,France,NaN,"€15,600",NaN,"€29,000",NaN,A,43.0,...,8,Management Acceleration Programme (MAP),120,29.0,75,Advanced Management Programme (AMP),NaN,160.0,19.0,58
3,University of Michigan: Ross,3,US,NaN,"$7,300",NaN,"$7,750",NaN,NaN,28.0,...,2,Strategic Leaders Program,34,30.0,70,The Positive Leader,NaN,29.5,44.0,25
4,Stanford Graduate School of Business,5,US,NaN,"$19,000",NaN,"$78,000",NaN,A,27.0,...,1,Stanford LEAD Online Business Program,333,33.0,"1,709",SEP (2019) (Stanford Executive Program),NaN,240.0,20.0,255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,Lagos Business School,69,Nigeria,NaN,"$22,563",NaN,"$44,165",NaN,C,37.0,...,8,Senior Management Programme,206,40.0,"1,487",Advanced Management Programme,NaN,163.0,27.0,124
62,Kedge Business School,71,France,NaN,"€12,100",NaN,"€6,200",NaN,C,38.0,...,67,Programme de Management Général,455,58.0,52,Strategical development for Health Organizations,NaN,36.0,64.0,73
63,Católica Lisbon School of Business and Economics,72,Portugal,NaN,"€6,250",NaN,"€9,500",NaN,C,43.0,...,7,General Management Program,143,57.0,81,Executive Program in Advanced Management,NaN,167.0,38.0,18
64,Solvay Brussels School of Economics and Manage...,73,Belgium,NaN,"€8,400",NaN,"€10,950",NaN,D,0.0,...,3,Accelerated Management Programme,144,37.0,214,Executive Master in Management,NaN,220.0,24.0,60


In [36]:
inner_worked = try_inner.to_excel('modified_2021_v1.xlsx')